In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import shutil
import random
from pathlib import Path

def analyze_image_size(image_path):
    """Анализирует размер изображения и возвращает нормализованные координаты bbox"""
    img = cv2.imread(image_path)
    if img is None:
        return (0.5, 0.5, 0.8, 0.8)  # fallback values
    
    height, width = img.shape[:2]
    # Предполагаем, что инструмент занимает центральную часть изображения
    bbox_width = 0.7 * width
    bbox_height = 0.7 * height
    x_center = width / 2
    y_center = height / 2
    
    # Нормализованные координаты для YOLO
    x_center_norm = x_center / width
    y_center_norm = y_center / height
    width_norm = bbox_width / width
    height_norm = bbox_height / height
    
    return (x_center_norm, y_center_norm, width_norm, height_norm)

def copy_images_with_annotations(src_dir, output_dir, class_idx, split, images, is_single_tool=True):
    """Копирует изображения и создает аннотации"""
    for img_name in images:
        try:
            # Копируем изображение
            src_img = os.path.join(src_dir, img_name)
            dst_img = os.path.join(output_dir, 'images', split, img_name)
            shutil.copy2(src_img, dst_img)
            
            # Создаем YOLO аннотацию
            txt_name = os.path.splitext(img_name)[0] + '.txt'
            
            if is_single_tool:
                # Для одиночных инструментов создаем bbox по центру
                bbox_coords = analyze_image_size(src_img)
                with open(os.path.join(output_dir, 'labels', split, txt_name), 'w') as f:
                    f.write(f"{class_idx} {bbox_coords[0]} {bbox_coords[1]} {bbox_coords[2]} {bbox_coords[3]}\n")
            else:
                # Для групповых и инструментов с линейкой - проверяем существующие аннотации
                annotation_path = os.path.join(src_dir, txt_name)
                if os.path.exists(annotation_path):
                    # Копируем существующую аннотацию
                    dst_txt = os.path.join(output_dir, 'labels', split, txt_name)
                    shutil.copy2(annotation_path, dst_txt)
                else:
                    # Создаем пустую аннотацию (требует ручной разметки)
                    dst_txt = os.path.join(output_dir, 'labels', split, txt_name)
                    open(dst_txt, 'w').close()
                    print(f"  ⚠ Создана пустая аннотация для {img_name} (требует ручной разметки)")
                        
        except Exception as e:
            print(f"❌ Ошибка при обработке {img_name}: {e}")

def process_single_tools(dataset_path, output_dir, classes):
    """Обрабатывает папки с отдельными инструментами"""
    print("\n=== Обработка отдельных инструментов ===")
    
    total_images = 0
    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        
        if not os.path.exists(class_path):
            print(f"⚠ Предупреждение: папка {class_path} не существует!")
            continue
            
        images = [f for f in os.listdir(class_path) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
        
        print(f"{class_name}: {len(images)} изображений")
        
        if len(images) == 0:
            print(f"⚠ Предупреждение: в папке {class_name} нет изображений!")
            continue
        
        # Разделяем на train/val
        if len(images) == 1:
            train_imgs = images
            val_imgs = []
        else:
            train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42, shuffle=True)
        
        print(f"  Train: {len(train_imgs)}, Val: {len(val_imgs)}")
        total_images += len(images)
        
        # Копируем изображения и создаем аннотации
        copy_images_with_annotations(class_path, output_dir, class_idx, 'train', train_imgs, is_single_tool=True)
        copy_images_with_annotations(class_path, output_dir, class_idx, 'val', val_imgs, is_single_tool=True)
    
    print(f"✓ Обработано отдельных инструментов: {total_images} изображений")
    return total_images

def process_tools_with_ruler(dataset_path, output_dir, ruler_folder, classes):
    """Обрабатывает инструменты с линейкой"""
    print(f"\n=== Обработка инструментов с линейкой: {ruler_folder} ===")
    
    ruler_path = os.path.join(dataset_path, ruler_folder)
    
    if not os.path.exists(ruler_path):
        print(f"⚠ Предупреждение: папка {ruler_path} не существует!")
        return 0
    
    # Создаем маппинг имен инструментов к классам
    class_mapping = {}
    for class_idx, class_name in enumerate(classes):
        class_mapping[class_name] = class_idx
    
    total_images = 0
    images = [f for f in os.listdir(ruler_path) 
             if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
    
    print(f"Найдено изображений с линейкой: {len(images)}")
    
    if len(images) == 0:
        print("⚠ В папке с линейкой нет изображений!")
        return 0
    
    # Разделяем на train/val
    if len(images) == 1:
        train_imgs = images
        val_imgs = []
    else:
        train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42, shuffle=True)
    
    # Определяем класс для каждого изображения по имени файла
    for split, imgs in [('train', train_imgs), ('val', val_imgs)]:
        for img_name in imgs:
            try:
                # Определяем класс инструмента по имени файла
                tool_class = None
                for class_name in classes:
                    if class_name.lower() in img_name.lower():
                        tool_class = class_mapping[class_name]
                        break
                
                if tool_class is None:
                    print(f"⚠ Не удалось определить класс для {img_name}")
                    continue
                
                # Копируем изображение
                src_img = os.path.join(ruler_path, img_name)
                dst_img = os.path.join(output_dir, 'images', split, img_name)
                shutil.copy2(src_img, dst_img)
                
                # Создаем аннотацию
                txt_name = os.path.splitext(img_name)[0] + '.txt'
                copy_images_with_annotations(ruler_path, output_dir, tool_class, split, [img_name], is_single_tool=False)
                
                total_images += 1
                
            except Exception as e:
                print(f"❌ Ошибка при обработке {img_name}: {e}")
    
    print(f"✓ Обработано инструментов с линейкой: {total_images} изображений")
    return total_images

def process_group_photos(dataset_path, output_dir, group_folders, classes):
    """Обрабатывает групповые фотографии инструментов"""
    print("\n=== Обработка групповых фото ===")
    
    total_images = 0
    for group_folder in group_folders:
        group_path = os.path.join(dataset_path, group_folder)
        
        if not os.path.exists(group_path):
            print(f"⚠ Предупреждение: папка {group_path} не существует!")
            continue
            
        images = [f for f in os.listdir(group_path) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
        
        print(f"{group_folder}: {len(images)} изображений")
        
        if len(images) == 0:
            print(f"⚠ Предупреждение: в папке {group_folder} нет изображений!")
            continue
        
        # Разделяем на train/val
        if len(images) == 1:
            train_imgs = images
            val_imgs = []
        else:
            train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42, shuffle=True)
        
        # Копируем изображения и аннотации
        for split, imgs in [('train', train_imgs), ('val', val_imgs)]:
            for img_name in imgs:
                try:
                    # Копируем изображение
                    src_img = os.path.join(group_path, img_name)
                    dst_img = os.path.join(output_dir, 'images', split, img_name)
                    shutil.copy2(src_img, dst_img)
                    
                    # Копируем аннотацию (если существует)
                    txt_name = os.path.splitext(img_name)[0] + '.txt'
                    annotation_path = os.path.join(group_path, txt_name)
                    
                    if os.path.exists(annotation_path):
                        dst_txt = os.path.join(output_dir, 'labels', split, txt_name)
                        shutil.copy2(annotation_path, dst_txt)
                        print(f"  ✓ Добавлена аннотация для {img_name}")
                    else:
                        # Создаем пустую аннотацию
                        dst_txt = os.path.join(output_dir, 'labels', split, txt_name)
                        open(dst_txt, 'w').close()
                        print(f"  ⚠ Создана пустая аннотация для {img_name} (требует ручной разметки)")
                    
                    total_images += 1
                    
                except Exception as e:
                    print(f"❌ Ошибка при обработке группового фото {img_name}: {e}")
    
    print(f"✓ Обработано групповых фото: {total_images} изображений")
    return total_images

def check_dataset_stats(output_dir):
    """Проверяет статистику подготовленного датасета"""
    print("\n=== Статистика датасета ===")
    
    train_images = len(os.listdir(os.path.join(output_dir, 'images', 'train')))
    val_images = len(os.listdir(os.path.join(output_dir, 'images', 'val')))
    train_labels = len(os.listdir(os.path.join(output_dir, 'labels', 'train')))
    val_labels = len(os.listdir(os.path.join(output_dir, 'labels', 'val')))
    
    print(f"✓ Тренировочные изображения: {train_images}")
    print(f"✓ Валидационные изображения: {val_images}")
    print(f"✓ Тренировочные аннотации: {train_labels}")
    print(f"✓ Валидационные аннотации: {val_labels}")
    print(f"✓ Общее количество изображений: {train_images + val_images}")
    
    # Проверяем количество объектов в аннотациях
    total_objects = 0
    for split in ['train', 'val']:
        labels_dir = os.path.join(output_dir, 'labels', split)
        for label_file in os.listdir(labels_dir):
            if label_file.endswith('.txt'):
                with open(os.path.join(labels_dir, label_file), 'r') as f:
                    objects = len(f.readlines())
                    total_objects += objects
    
    print(f"✓ Всего объектов в аннотациях: {total_objects}")

def prepare_yolo_dataset(dataset_path, output_dir='yolo_dataset'):
    """
    Подготавливает датасет в формате YOLO для всех типов изображений
    """
    # Получаем все папки
    all_folders = sorted([d for d in os.listdir(dataset_path) 
                         if os.path.isdir(os.path.join(dataset_path, d))])
    
    print(f"Найдено папок: {len(all_folders)}")
    print("Все папки:", all_folders)
    
    # Создаем директории
    for split in ['train', 'val']:
        os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)
    
    # Определяем типы папок
    tools_folders = [folder for folder in all_folders 
                    if 'групповые' not in folder.lower() 
                    and 'линейк' not in folder.lower()
                    and 'group' not in folder.lower()
                    and 'ruler' not in folder.lower()]
    
    ruler_folders = [folder for folder in all_folders 
                    if 'линейк' in folder.lower() 
                    or 'ruler' in folder.lower()]
    
    group_folders = [folder for folder in all_folders 
                    if 'групповые' in folder.lower() 
                    or 'group' in folder.lower()]
    
    print(f"✓ Папки с инструментами: {len(tools_folders)}")
    print("Инструменты:", tools_folders)
    print(f"✓ Папки с линейкой: {len(ruler_folders)}")
    print("Линейка:", ruler_folders)
    print(f"✓ Групповые папки: {len(group_folders)}")
    print("Групповые:", group_folders)
    
    # Сохраняем классы инструментов
    with open(os.path.join(output_dir, 'classes.txt'), 'w') as f:
        for i, class_name in enumerate(tools_folders):
            f.write(f"{class_name}\n")
    
    # Обрабатываем все типы данных
    single_count = process_single_tools(dataset_path, output_dir, tools_folders)
    ruler_count = 0
    for ruler_folder in ruler_folders:
        ruler_count += process_tools_with_ruler(dataset_path, output_dir, ruler_folder, tools_folders)
    
    group_count = process_group_photos(dataset_path, output_dir, group_folders, tools_folders)
    
    # Проверяем результат
    check_dataset_stats(output_dir)
    
    print(f"\n📊 ИТОГОВАЯ СТАТИСТИКА:")
    print(f"  Одиночные инструменты: {single_count}")
    print(f"  Инструменты с линейкой: {ruler_count}")
    print(f"  Групповые фото: {group_count}")
    print(f"  Всего: {single_count + ruler_count + group_count}")
    
    return tools_folders

def create_dataset_config(output_dir='yolo_dataset', output_yaml='dataset.yaml'):
    """
    Создает конфигурационный файл для датасета YOLO
    """
    # Читаем классы из файла
    classes_path = os.path.join(output_dir, 'classes.txt')
    if not os.path.exists(classes_path):
        print(f"Ошибка: файл {classes_path} не найден!")
        return None, None
    
    with open(classes_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    
    # Создаем словарь с конфигурацией
    config = {
        'path': os.path.abspath(output_dir),
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(classes),
        'names': {i: class_name for i, class_name in enumerate(classes)}
    }
    
    # Сохраняем в YAML файл
    with open(output_yaml, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, allow_unicode=True)
    
    print(f"✓ Создан конфиг файл: {output_yaml}")
    print(f"✓ Путь к данным: {os.path.abspath(output_dir)}")
    print(f"✓ Количество классов: {len(classes)}")
    print(f"✓ Классы: {classes}")
    return config, classes

def train_yolo_model(output_dir='yolo_dataset', model_size='s', epochs=100):
    """
    Обучение модели YOLOv8 с улучшенными параметрами
    """
    # Создаем конфигурационный файл
    config, classes = create_dataset_config(output_dir)
    
    if config is None:
        raise ValueError("Не удалось создать конфигурационный файл")
    
    # Загружаем предобученную модель
    print(f"🚀 Загрузка модели YOLOv8{model_size}.pt...")
    model = YOLO(f'yolov8{model_size}.pt')
    
    # Расширенные параметры обучения
    train_params = {
        'data': 'dataset.yaml',
        'epochs': epochs,
        'imgsz': 640,
        'batch': 16,
        'name': f'yolov8{model_size}_tools_detection',
        'patience': 20,
        'optimizer': 'AdamW',
        'lr0': 0.001,
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'augment': True,
        'hsv_h': 0.015,
        'hsv_s': 0.7,
        'hsv_v': 0.4,
        'degrees': 45.0,
        'translate': 0.1,
        'scale': 0.5,
        'shear': 0.0,
        'perspective': 0.0,
        'flipud': 0.0,
        'fliplr': 0.5,
        'mosaic': 1.0,
        'mixup': 0.1,
        'copy_paste': 0.1,
        'erasing': 0.4,
        'dropout': 0.1,
        'val': True,
        'save': True,
        'save_period': 10,
        'device': 'cpu',  # Можно изменить на 'cuda' или 0 для GPU
        'workers': 8,
        'single_cls': False,
        'verbose': True,
        'exist_ok': True
    }
    
    print("🎯 Начинаем обучение модели...")
    print(f"📊 Параметры обучения: {epochs} эпох, размер батча: 16")
    print(f"📁 Данные: {config['path']}")
    
    # Обучаем модель
    results = model.train(**train_params)
    
    print("✅ Обучение завершено!")
    return model, results

def export_model_to_onnx(model, output_path='yolov8_tools.onnx'):
    """
    Экспорт модели в формат ONNX с оптимизацией
    """
    print("📤 Экспорт модели в ONNX...")
    
    # Получаем путь к лучшей модели
    model_path = model.trainer.best if hasattr(model.trainer, 'best') else 'runs/detect/yolov8s_tools_detection/weights/best.pt'
    
    if os.path.exists(model_path):
        # Загружаем лучшую модель для экспорта
        best_model = YOLO(model_path)
        best_model.export(format='onnx', imgsz=640, simplify=True, dynamic=True, opset=12)
        print(f"✅ Модель экспортирована в: {output_path}")
    else:
        # Экспортируем текущую модель
        model.export(format='onnx', imgsz=640, simplify=True, dynamic=True, opset=12)
        print(f"✅ Модель экспортирована в: {output_path}")

def evaluate_model(model, data_path='yolo_dataset'):
    """
    Оценка модели на тестовых данных
    """
    print("📊 Оценка модели...")
    
    try:
        metrics = model.val(data=os.path.join(data_path, 'dataset.yaml'), split='val')
        
        print("📈 Результаты оценки:")
        print(f"  mAP50: {metrics.box.map50:.4f}")
        print(f"  mAP50-95: {metrics.box.map:.4f}")
        print(f"  Precision: {metrics.box.mp:.4f}")
        print(f"  Recall: {metrics.box.mr:.4f}")
        
        return metrics
    except Exception as e:
        print(f"❌ Ошибка при оценке модели: {e}")
        return None

def predict_on_image(model, image_path, conf_threshold=0.25, iou_threshold=0.45):
    """
    Предсказание на одном изображении с улучшенной визуализацией
    """
    # Проверяем существование файла
    if not os.path.exists(image_path):
        print(f"❌ Изображение не найдено: {image_path}")
        return None
    
    # Выполняем предсказание
    results = model(image_path, conf=conf_threshold, iou=iou_threshold, augment=False)
    
    # Визуализируем результаты
    for i, result in enumerate(results):
        # Рисуем bounding boxes с улучшенной визуализацией
        img = result.plot(line_width=2, font_size=1.0, conf=True, labels=True)
        
        # Создаем окно с фиксированным размером
        height, width = img.shape[:2]
        max_display_size = 1200
        if max(height, width) > max_display_size:
            scale = max_display_size / max(height, width)
            new_width = int(width * scale)
            new_height = int(height * scale)
            img = cv2.resize(img, (new_width, new_height))
        
        # Показываем изображение
        cv2.imshow('Инструменты - обнаружение', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        # Детальная информация об обнаруженных объектах
        print(f"\n🔍 Результаты обнаружения для изображения {i+1}:")
        if len(result.boxes) > 0:
            for j, box in enumerate(result.boxes):
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                bbox = box.xyxy[0].cpu().numpy()
                class_name = model.names[class_id]
                print(f"  Объект {j+1}: {class_name} "
                      f"(уверенность: {confidence:.3f}) "
                      f"BBox: {bbox.astype(int)}")
        else:
            print("  Объекты не обнаружены")
    
    return results

def predict_on_folder(model, folder_path, conf_threshold=0.25):
    """
    Предсказание на всех изображениях в папке
    """
    if not os.path.exists(folder_path):
        print(f"❌ Папка не найдена: {folder_path}")
        return
    
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    images = [f for f in os.listdir(folder_path) if f.lower().endswith(image_extensions)]
    
    print(f"🔍 Найдено {len(images)} изображений в папке {folder_path}")
    
    for i, img_name in enumerate(images):
        img_path = os.path.join(folder_path, img_name)
        print(f"\n📄 Обработка {i+1}/{len(images)}: {img_name}")
        predict_on_image(model, img_path, conf_threshold)

def main():
    # Пути к данным
    dataset_path = '/data/vscode/HacatonAeroflot/Aeroflot-project/datasets/raw'
    output_dir = '/data/vscode/HacatonAeroflot/Aeroflot-project/yolo_dataset'
    
    try:
        # Проверяем существование пути
        if not os.path.exists(dataset_path):
            print(f"❌ Ошибка: путь {dataset_path} не существует!")
            return
        
        print("=" * 60)
        print("🛠️  СИСТЕМА ОБНАРУЖЕНИЯ ИНСТРУМЕНТОВ (УЛУЧШЕННАЯ)")
        print("=" * 60)
        
        # Шаг 1: Подготовка датасета
        print("\n📁 ШАГ 1: Подготовка датасета...")
        classes = prepare_yolo_dataset(dataset_path, output_dir)
        
        # Шаг 2: Создание конфигурации
        print("\n⚙️  ШАГ 2: Создание конфигурации...")
        config, classes = create_dataset_config(output_dir)
        
        if config is None:
            raise ValueError("Не удалось создать конфигурацию датасета")
        
        # Шаг 3: Обучение модели
        print("\n🎓 ШАГ 3: Обучение модели...")
        model, results = train_yolo_model(output_dir, model_size='s', epochs=100)
        
        # Шаг 4: Сохранение модели
        print("\n💾 ШАГ 4: Сохранение модели...")
        model.save('best_tools_detection.pt')
        print("✅ Модель сохранена как 'best_tools_detection.pt'")
        
        # Шаг 5: Экспорт в ONNX
        print("\n📤 ШАГ 5: Экспорт в ONNX...")
        export_model_to_onnx(model)
        
        # Шаг 6: Оценка модели
        print("\n📊 ШАГ 6: Оценка модели...")
        metrics = evaluate_model(model, output_dir)
        
        # Шаг 7: Тестирование на разных типах изображений
        print("\n🧪 ШАГ 7: Тестирование модели на разных типах изображений...")
        
        test_types = ['val']  # Можно добавить 'train' для большего количества тестов
        
        for split in test_types:
            test_dir = os.path.join(output_dir, 'images', split)
            if os.path.exists(test_dir):
                test_images = os.listdir(test_dir)
                if test_images:
                    # Тестируем на нескольких изображениях
                    for i, test_img in enumerate(test_images[:3]):  # Первые 3 изображения
                        test_image_path = os.path.join(test_dir, test_img)
                        print(f"\n🔍 Тест {i+1}: {test_img}")
                        predict_on_image(model, test_image_path)
        
        print("\n" + "=" * 60)
        print("✅ ВСЕ ЭТАПЫ ЗАВЕРШЕНЫ УСПЕШНО!")
        print("=" * 60)
        
    except Exception as e:
        print(f"\n❌ КРИТИЧЕСКАЯ ОШИБКА: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()